In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
%run ./Project2_V2_0211.ipynb

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


NameError: name 'combined_table_2' is not defined

NameError: name 'combined_table_2' is not defined

# The code below is to analyze the break-even point of revenue and cost of this DR Program

Model:

n is the number of household who join in the DR program(based on data)

p_energy is the energy used for heating that was original supplied by gas but now supplied by the curtailed energy  (per household) (based on data)

P1 is the price that we buy the curtailed energy from the generators (wholesale price, based on online data)

P2 is the price that we sell the curtailed energy to the local, which are used for heating (estimated)

P3 is the price of gas used to supply the heater (based on online data)

FC is the installment cost of this DR program (to be determined)

VC is the variable cost per household of this PR program (to be determined)


Revenue for generator: n * p_energy  * P1 

Revenue for each household:  p_energy  * (P3 - P2) 

Financial benefits to Kaluza: Revenue - cost =  n * p_energy  * (P2-P1) -  FC - VC * n

Savings to the Community: when the demand and supply are not balanced, it will hurt the grid. Therefore, we need to balance the usage of electronicty during peak and offpeak time and when wind is strong and not strong. 

In [105]:
combined_table_2.head()

,Timestamp_H,Demand_mean_kw,N_households,res_heating_perc,curtailment(kwh),Year,Month,Hour,Minute,household_with_DR,addi_demand_by_DR,saved_curtailment
0,2017-01-01 00:00:00,0.220106,5428,0.526316,0.0,2017,1,0,0,814.0,47.149126,0.0
1,2017-01-01 00:30:00,0.205945,5429,0.403158,0.0,2017,1,0,30,814.0,33.792558,0.0
2,2017-01-01 01:00:00,0.189090,5429,0.280000,0.0,2017,1,1,0,814.0,21.548684,0.0
3,2017-01-01 01:30:00,0.173118,5429,0.248696,0.0,2017,1,1,30,814.0,17.522860,0.0
4,2017-01-01 02:00:00,0.159051,5429,0.217391,0.0,2017,1,2,0,814.0,14.072525,0.0


In [189]:
# Read the wholesale price for electricity 
col_list = ['YEAR','MONTH', 'WHOLESALE PRICE(￡/MWh）']
wholesale_price = pd.read_csv('wholesale_price.csv',usecols=col_list) # https://www.businesselectricityprices.org.uk/retail-versus-wholesale-prices/
wholesale_price['Cumulative_Month'] = wholesale_price.index.values + 1
wholesale_price

,YEAR,MONTH,WHOLESALE PRICE(￡/MWh）,Cumulative_Month
0,16,1,36.95,1
1,16,2,36.95,2
2,16,3,36.95,3
3,16,4,34.37,4
4,16,5,34.37,5
5,16,6,34.37,6
6,16,7,38.40,7
7,16,8,38.40,8
8,16,9,38.40,9
9,16,10,56.03,10


In [3]:
# Define the paramaters used in the models which analyze the financial and social beneifts 
n = combined_table_2.groupby(['Month'])['household_with_DR'].mean() # the number of households who join in the DR program(based on calculation)
p_energy = combined_table_2.groupby(['Month'])['household_with_DR'].sum() # is the energy used for heating that was original supplied by gas but now supplied by the curtailed energy  (per household) (based on calculation)
P1 = wholesale_price.loc[wholesale_price.YEAR == 17]
            # The unit is (GBP/KWh)
            # the wholesale price is the generator sells the energy in the general market (based on online data)
                
P2 = 7.5 * 0.01 # The unit is (GBP/KWh)
            # There is the gas price which the local residents are used for heating
            # https://www.thegreenage.co.uk/cheaper-heat-home-gas-electricity/
P3 = 15 * 0.01 # The unit is (GBP/KWh)
            # This is the electricity price that the local residents will use for heating  
            # https://www.thegreenage.co.uk/cheaper-heat-home-gas-electricity/
        
FC = 100 # The unit is (GBP/Household)
         # the installment cost of this DR program 
VC = 1 # The unit is (GBP/month * household） 


def return_Revenue_for_generator(n,p_energy,P1): 
    return n*p_energy*P1


def return_Loss_for_all_household(p_energy,P3,P2,n):
    return p_energy*(P2-P3)



def return_Financial_benefits_to_Kaluza(n,p_energy,P2,P1,FC,VC):
    return n * p_energy * (P2-P1) -  FC - VC * n



def return_Savings_to_the_Community():
    # curtailed energy * wholesale price + (gas heating use - retail electricity price) * mwh - fixed cost for installation 
    # unfinished
    return 

NameError: name 'combined_table_2' is not defined

In [4]:
month = combined_table_2.Month.unique()
table_Q2 = pd.DataFrame(data = month,columns=['Month']).set_index('Month')
table_Q2['n'] = n
table_Q2['p_energy(KWh)'] = p
table_Q2['P1(￡/KWh)'] = wholesale_price['WHOLESALE PRICE(￡/MWh）'] * 0.001
table_Q2['P3(￡/KWh)'] = P3
table_Q2['P2(￡/KWh)'] = (table_Q2['P3(￡/KWh)']+table_Q2['P1(￡/KWh)'])/2 
table_Q2['FC(￡)'] = FC
table_Q2['VC(￡)'] = VC
table_Q2['Revenue_Generator'] = table_Q2.apply(lambda row:return_Revenue_for_generator(row['n'],row['p_energy(KWh)'],row['P1(￡/KWh)']),axis=1)
table_Q2['Loss_All_Household'] = table_Q2.apply(lambda row:return_Loss_for_all_household(row['p_energy(KWh)'],row['P3(￡/KWh)'],row['P2(￡/KWh)'],row['n']),axis=1)
table_Q2['Revenue_minus_Loss'] = table_Q2['Revenue_Generator'] + table_Q2['Loss_All_Household']
table_Q2['Net_Income_for_Generator'] = table_Q2['Revenue_minus_Loss']/2
table_Q2['Net_Income_for_Kaluza'] = table_Q2['Revenue_minus_Loss']/2
table_Q2['Fixed_Cost_Kaluza'] = 0
table_Q2['Fixed_Cost_Kaluza'].iloc._setitem_with_indexer(0, FC * table_Q2['n'].iloc[0])
table_Q2['Variable_Cost_Kaluza'] = VC * table_Q2['n']
table_Q2['Total_cost_Kaluza'] = table_Q2['Fixed_Cost_Kaluza'] + table_Q2['Variable_Cost_Kaluza']
table_Q2['Net_Cash_flow_Kaluza'] = table_Q2['Net_Income_for_Kaluza']- table_Q2['Total_cost_Kaluza']
table_Q2

NameError: name 'combined_table_2' is not defined

# Q1: How much energy is currently curtailed annually across Orkney Isles?

### Step 1: 
To find max  curtailment from wind generator point of view
i.e. find difference betwee setpoint and power_kw_y. 
#This is the max amount of curtailment without customer consumption

Steps:
    1. curtailed energy per timestamp = Max(Power_kw_y - setpoint,0) => result_1 (done!)
    2 calculate energy curtailment based on historical_max_power_kw_-current_power_kw if setpoint>900 (to do by arianna &qian-aim for tonight   done!)
    3. aggregate the dataset to half hourly dataset (to do by arianna &qian-aim for tonight-done!)
    4. Calculate the yearly curtailed energy (to do by arianna &qian-aim for tonight-done!)
    
 

# Q2: How much can this be reduced by different levels of DR penetration?

### in addtion, Step 2 to include customer and 40MW export
To find available curtailment with customer demand involved. This is max amount of curtailment can be saved with extreme high DR penetration! 

i.e. new curtailment = result_1 - total demand - 40 (if there is any left_over, this is possible curtailment)

Steps: 
    1. Total customer demand = N households*Demand_mean_kw (to do by arianna &qian - aim for tonight - done!)  
    
    2. curstomer involved curtailment = Max(result_1 - 40 - total customer demand,0)  => call that result_2 (to do by arianna &qian-aim for tonight - we calculated the annual total)
        2.1 finding: curtailed energy = 26.6% of total energy consumption
        2.2 finding: total customer electricity consumption increase from 5-ish pm onwards, this is also the time that wind farm most likely been uncurtailed - see powerbi.
    



# Q3:How many local households would need to be on our DR scheme in order to supply this level of DR?

### in addition, Step 3 to include heating period in

#Note that the company have control of customer's heater, hence it's not on customer's side to determine whether he want to switch the control on or not

#As such, the company will choose the heating period that depend on time period that has maximum daily curtailed energy loss. This is because he he only have one year of customer's demand data and as such can't anticipate exactly what customer is going to demand in each day)

####  Note: We can further break it down to different seasonality, weekday/weekend, or at daily level for further steps!

Steps:
    1. search average heating time for hot water system, search for DR system MW 
    2. Plot avearge intraday curtailed energy => convert result_2 to average amount
    3. Calculate curtailed energy by split up result_2 by different charging window, and amount of customer!-- We need to align on how to deal with this problem!
        My thoughts:
            1. Assume customer penetration 10%, given charging window 2hour(as example), use the charging window from  previous step to calculate energy save that could potentionally be curtailed.
            2. What if we change customer penetration to 20%? How about 30%? How much energy can the company save now?
